<a href="https://colab.research.google.com/github/hurgj123kr/Project/blob/main/AI_05_%E1%84%92%E1%85%A5%E1%84%89%E1%85%A5%E1%86%BC%E1%84%92%E1%85%A9_Section1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  다음분기의 게임 데이터셋을 불러옵니다.
### 1)결측치가 있다면 제거해주고
### 2)연도별로 정렬해줍니다.
### 3)각 지역별 출고값열들 특정 문자 제거후 타입변경
#### 4)이상한 연도들 원래 연도값으로 변경해주기

In [ ]:
import pandas as pd
import numpy as np
#!sudo apt-get install -y fonts-nanum
#!sudo fc-cache -fv
#!rm ~/.cache/matplotlib -rf
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
plt.rc('axes', unicode_minus=False)
url ='https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv'
df = pd.read_csv(url,thousands=',')
#데이터 확인
df.columns
df.shape
#데이터에 이상이없는지 확인합니다
df.isnull().sum()
##데이터에 문제가있으므로 제거해줍니다.
df.dropna(inplace=True)
df.isnull().sum()
#이제 연도별로 정렬해줍니다.
df.sort_values(by=['Year'], axis=0,inplace=True)
df.drop(['Unnamed: 0'], axis=1,inplace=True)
df = df.replace({'K':'', 'M':''}, regex=True)
df = df.astype({'NA_Sales':'float','EU_Sales':'float','JP_Sales':'float',
                          'Other_Sales':'float'})
df.dtypes
df

In [ ]:
#연도 이상치 고치기
year = df['Year'].unique()
temp = []
for i in year:
    if i <= 1970:
        print(i)
        temp.append(i)
df.loc[(df['Year'].values <= 70),'Year' ] += 2000
df.loc[((df['Year'].values >=70) & (df['Year'].values <= 100)),'Year'] += 1900
df['Year'].value_counts()
df

# 이제 불러온 데이터를 통해서 지역에따라 장르의 선호도


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
#각 장르별 나라들의 출고값을 합산한것입니다.
game_genre = df[['Genre','NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]
game_genre.columns =['장르','북미지역 출고량','유렵지역 출고량','일본지역 출고량','다른지역 출고량']
genre_group = game_genre.groupby(['장르']).sum()
game_df = genre_group.reset_index()
#확인
game_df
game_graph = pd.melt(game_df, id_vars=['장르'], value_vars=game_df.columns[1:],
                      var_name='판매지역', value_name='출고량')
game_graph.head()
plt.figure(figsize=(14, 13))
sns.barplot(data=game_graph, x='장르', y='출고량', hue='판매지역')
plt.title('지역별 장르 선호도', loc='center', fontsize=25)
plt.xticks(fontsize=13)
plt.yticks(fontsize=18)
plt.xlabel('장르', fontsize=20)
plt.ylabel('출고량', fontsize=20)    
plt.legend(fontsize=20)
plt.show()

### 각 지역별 출고량이 높은 5가지 장르 그래프
## 그래프 결과 북미지역은 액션게임이 유렵지역또한 액션게임 일본은 롤플레잉게임이 다른지역들은 액션게임이
#### 게임시장은 액션게임이 유행하는듯하다.

In [ ]:
#국가별 출고량 TOP5
def top5_genre(df, sales_col):
    new_df = game_df.loc[:, ['장르', sales_col]].sort_values(by=sales_col, ascending=False).reset_index(drop=True).head(5)
    new_df.columns = ['장르', '출고량']
    return new_df
# Top5 데이터 저장
na_genre_top5 = top5_genre(game_df, '북미지역 출고량')
eu_genre_top5 = top5_genre(game_df, '유렵지역 출고량')
jp_genre_top5 = top5_genre(game_df, '일본지역 출고량')
other_genre_top5 = top5_genre(game_df, '다른지역 출고량')
#데이터 리스트에 담기
data_list = [na_genre_top5, eu_genre_top5, jp_genre_top5, other_genre_top5]
columns_list = game_df.columns[1:]
#그래프 그리기
fig, axs = plt.subplots(figsize=(26, 10), nrows=1, ncols=4)
for col, i, data in zip(columns_list, range(len(columns_list)), data_list):
    axs[i].set_title(col + ' ' + 'TOP5', fontsize=24)
    sns.barplot(x='장르', y='출고량', data=data, ax=axs[i])
    axs[i].tick_params(axis='both',labelsize=14)
    axs[i].set_xlabel('장르', fontsize=16)
    axs[i].set_ylabel('출고량', fontsize=16)
plt.show()
import matplotlib.pyplot as plt

## 출고량에 따른 연도별 트랜드구하기

### 그래프에 따르면 연도별로 Action게임이 제일 트랜드했던것같다.

In [ ]:
#각 연도에 나온 장르들의 게임 갯수
year_genre = df.groupby(['Year','Genre']).size().reset_index(name='number')
year_genre
#각 연도에 나온장르별 많은 갯수
year_max = year_genre.groupby(['Year'])['number'].transform(max) == year_genre['number']
#확인
year_max
#year_max 인덱스를 year_genre 인덱스로 바꿔준것.
year_max_genre = year_genre[year_max].reset_index(drop=True)
year_max_genre
#연도와 number안에있는 중복값을 없애줘서 내가 알고싶은 제일높은 갯수 마지막만 나오게끔
year_max_genre = year_max_genre.drop_duplicates(subset=['Year','number'], keep='last').reset_index(drop=True)
year_max_genre.head()
#column명 한글로바꾸기
year_max_genre.columns = ['년도', '장르', '출고량']
year_max_genre
#전지역의 출고량을 합쳐서 분석을 좀더 정확하게 하기
global_sales = []
for i in df.index:
    df_sales = df.loc[i, 'NA_Sales' : 'Other_Sales'].sum()
    global_sales.append(df_sales)
df['Global_Sales'] = global_sales
df.head(10)
#연도별 많은 출고량을 달성한 데이터프레임
year_max_sales = df.groupby(['Year', 'Genre'])['Global_Sales'].sum().reset_index()
con = year_max_sales['Global_Sales'] == year_max_sales.groupby(['Year'])['Global_Sales'].transform(max)
year_max_sales = year_max_sales[con]
year_max_sales.columns = ['연도', '장르', '출고량']
# 장르 데이터 저장
genre = year_max_sales['장르'].values
# 그래프 그리기
plt.figure(figsize=(28,10))
ax = sns.barplot(x='년도', y='출고량', data=year_max_genre)
count = 0
for value in year_max_genre['출고량']:
    ax.text(x= count, y= value + 5, s=str(genre[count] + '---' + ' ' + str(value)),
            color='blue', size=20, rotation=90, ha='center')
    count = count + 1
plt.xticks(fontsize=12)
plt.yticks(fontsize=18)    
plt.xlabel('연도', fontsize=20)
plt.ylabel('출고량', fontsize=20)
ax.set_title('연도별 최고 출고량을 기록한 장르', fontsize=25, y=1.0, loc='center')
plt.show()

#출고량이 높은 게임에 대한 분석
### 분석결과 1998년 닌텐도64기기 이면서 액션게임이 일등이다.
### 다음 2위인 슈퍼마리오브로스3인 플랫폼게임이다.
### 파이널판타지9 가 3위를 차지했다.

In [ ]:
#게임명과 전지역의 출고량 에따른 분석
top5 = df.loc[df.Global_Sales.sort_values(ascending=False).index].reset_index(drop=True).head(5)
rank = [str(x)+' 위' for x in range(1, 6)]
top5['rank'] = rank
top5
#그래프 그리기
plt.figure(figsize=(32,10))
a = sns.barplot(x='rank', y='Global_Sales', data=top5)
i = 0
for name, year, val, platform, genre in zip(top5['Name'],top5['Year'], top5['Global_Sales'], 
                                                top5['Platform'],top5['Genre']):
    a.text(x=i, y=val-600, s=(name) , color='black', ha='center', fontsize=15)
    a.text(x=i, y=val-400, s=(str(year) + '년' + '\n' + ' 장르: ' + genre + '\n' + ' 플랫폼: ' + platform), color='black', ha='center', fontsize=15)
    i+=1
a.set_title('출고량 역대 TOP5 게임', fontsize=25)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel('게임명', fontsize=20)
plt.ylabel('출고량', fontsize=20)
plt.show()

# 출고량이 낮은 게임의 장르


*   북미쪽은 퍼즐,어드벤쳐,전략.격투,롪를레잉 게임이 인기가 없다.
*   유럽쪽은 퍼즐,전략,어드벤쳐,롤플레잉,레이싱순으로 인기가없는 장르다.

*   일본쪽은 퍼즐,슈팅,어드벤쳐,전략,격투장르가 인기가없다.
*   다른지역은 퍼즐,어드벤쳐,시뮬레이션,격투,isc라는 장르가 인기가없다

*   이결과로 보면 알수있듯이 전세계적으로 퍼즐 어드벤쳐 게임은 인기가 없다. 이 두개의 장르로 만들면 안된다.












In [ ]:
# 국가별 장르별 출고량 worst5
def worst5_genre(df, sales_col):
    new_df = game_df.loc[:, ['장르', sales_col]].sort_values(by=sales_col, ascending=True).reset_index(drop=True).head(5)
    new_df.columns = ['장르', '출고량']
    return new_df
# Top5 데이터 저장
na_genre_worst5 = worst5_genre(game_df, '북미지역 출고량')
eu_genre_worst5 = worst5_genre(game_df, '유렵지역 출고량')
jp_genre_worst5 = worst5_genre(game_df, '일본지역 출고량')
other_genre_worst5 = worst5_genre(game_df, '다른지역 출고량')
#데이터 리스트에 담기
data_list = [na_genre_worst5, eu_genre_worst5, jp_genre_worst5, other_genre_worst5]
columns_list = game_df.columns[1:]
#그래프 그리기
fig, axs = plt.subplots(figsize=(35, 10), nrows=1, ncols=4)
for col, i, data in zip(columns_list, range(len(columns_list)), data_list):
    axs[i].set_title(col + ' ' + 'worst5', fontsize=24)
    sns.barplot(x='장르', y='출고량', data=data, ax=axs[i])
    axs[i].tick_params(axis='both',labelsize=16)
    axs[i].set_xlabel('장르', fontsize=16)
    axs[i].set_ylabel('출고량', fontsize=16)
plt.show()
import matplotlib.pyplot as plt

# 무슨 플랫폼이 인기가 많은지 알아보자. 

### 각 지역별로 판매된는 플랫폼을 보았다. 유럽지역은 ps3가압도적으로 많다. 북미지역은 ps를 많이 구매햇다.


In [ ]:
#각 지역의 플랫폼 판매량 데이터프레임
platform = df[['Platform','NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]
platform.columns =['플랫폼','북미지역 출고량','유렵지역 출고량','일본지역 출고량','다른지역 출고량']
platform_group = platform.groupby(['플랫폼']).sum()
platform_df = platform_group.reset_index()
#확인
platform_df
platform_graph = pd.melt(platform_df, id_vars=['플랫폼'], value_vars=game_df.columns[1:],
                      var_name='판매지역', value_name='출고량')
#그래프 데이터프레임 값확인
platform_graph.head()
plt.figure(figsize=(16, 15))
sns.barplot(data=platform_graph, x='플랫폼', y='출고량', hue='판매지역')
plt.title('지역별 플랫폼 선호도', loc='center', fontsize=20)
plt.legend(fontsize=20)
plt.xticks(fontsize=12)
plt.yticks(fontsize=20)
plt.xlabel('플랫폼', fontsize=20)
plt.ylabel('출고량', fontsize=20)
plt.show()

# 플랫폼 top5 출고량 확인하기
## 전지역적으로 플스 플랫폼이 가장 많이 출고되었다.

In [ ]:
# 국가별 플랫폼별 출고량 TOP5
def top5_platform(df, sales_col):
    new_df = platform_df.loc[:, ['플랫폼', sales_col]].sort_values(by=sales_col, ascending=False).reset_index(drop=True).head(5)
    new_df.columns = ['플랫폼', '출고량']
    return new_df
#플랫폼  Top5 데이터 저장
na_platform_top5 = top5_platform(platform_df, '북미지역 출고량')
eu_platform_top5 = top5_platform(platform_df, '유렵지역 출고량')
jp_platform_top5 = top5_platform(platform_df, '일본지역 출고량')
other_platform_top5 = top5_platform(platform_df, '다른지역 출고량')
#각 지역별 데이터 리스트에 담기
data_list = [na_platform_top5,eu_platform_top5,jp_platform_top5,other_platform_top5]
columns_list = platform_df.columns[1:]
#그래프 그리기
fig, axs = plt.subplots(figsize=(35, 10), nrows=1, ncols=4)
for col, i, data in zip(columns_list, range(len(columns_list)), data_list):
    axs[i].set_title(col + ' ' + 'top5', fontsize=24)
    sns.barplot(x='플랫폼', y='출고량', data=data, ax=axs[i])
    axs[i].tick_params(axis='both',labelsize=16)
    axs[i].set_xlabel('플랫폼', fontsize=16)
    axs[i].set_ylabel('출고량', fontsize=16)
plt.show()
import matplotlib.pyplot as plt

# 연도별 최고출고량을 기록한 플랫폼
## 확인 결과 2008년에 ps3가 가장 많이 출고되었다.

In [ ]:
#각 연도에 나온 플랫폼들의 게임 갯수
year_platform = df.groupby(['Year','Platform']).size().reset_index(name='number')
year_platform
#각 연도에 나온 플랫폼별 많이 나왔던 갯수
platform_max = year_platform.groupby(['Year'])['number'].transform(max) == year_platform['number']
#확인
platform_max
#platform_max 인덱스를 year_platform 인덱스로 바꿔준것.
year_max_platform = year_platform[platform_max].reset_index(drop=True)
year_max_platform
#연도와 number안에있는 중복값을 없애줘서 내가 알고싶은 제일높은 갯수 마지막만 나오게끔
year_max_platform = year_max_platform.drop_duplicates(subset=['Year','number'], keep='last').reset_index(drop=True)
year_max_platform.head()
#column명 한글로바꾸기
year_max_platform.columns = ['년도', '플랫폼', '출고량']
year_max_platform
#연도별 많은 출고량을 달성한 플랫폼 df
year_max_sales = df.groupby(['Year', 'Platform'])['Global_Sales'].sum().reset_index()
con = year_max_sales['Global_Sales'] == year_max_sales.groupby(['Year'])['Global_Sales'].transform(max)
year_max_sales = year_max_sales[con]
year_max_sales.columns = ['년도', '플랫폼', '출고량']
# 플랫폼 데이터 저장
platform = year_max_sales['플랫폼'].values
# 그래프 그리기
plt.figure(figsize=(28,10))
ax = sns.barplot(x='년도', y='출고량', data=year_max_platform)
count = 0
for value in year_max_platform['출고량']:
    ax.text(x= count, y= value + 5, s=str(platform[count] + '---' + ' ' + str(value)),
            color='black', size=20, rotation=90, ha='center')
    count = count + 1
plt.yticks(fontsize=18)    
plt.xticks(fontsize=12)
plt.xlabel('연도', fontsize=20)
plt.ylabel('출고량', fontsize=20)
ax.set_title('연도별 최고 출고량을 기록한 플랫폼', fontsize=25, y=1.0, loc='center')
plt.show()

# 최근 10년간 많이 팔린 게임플랫폼,장르의 출고량
### ps플랫폼과 슈팅 장르가 많이 최근에 출고가 되었는데 최근 10년간의 많이 팔린 게임을 확인해봐야한다.

In [ ]:
def recent_year_Top5_sales(df, col, sales):
    return df.groupby(col)[sales].sum().reset_index(name='sales').sort_values('sales', ascending=False).head(5)
#최근 10년간의 데이터
year_sales_top_game = df[df['Year'] >= 2011].sort_values('Global_Sales', ascending=False).head(10)
year_sales_top_game = year_sales_top_game.reset_index(drop=True)
year_sales_top_game
#최근 10년간의 플랫폼별 출고량
platform_sales_top5 = recent_year_Top5_sales(year_sales_top_game, 'Platform', 'Global_Sales')
#최근 10년간의 장르별 출고량
genre_sales_top5 = recent_year_Top5_sales(year_sales_top_game, 'Genre', 'Global_Sales')

data_list = [platform_sales_top5, genre_sales_top5]
titles = ['최근 10년 판매량 1위 플랫폼 Top5', '최근 10년 판매량 1위 장르 Top5']
x_labels = ['플랫폼', '장르']

fig, axs = plt.subplots(figsize=(25,10), nrows=1, ncols=2)
for i, data, title in zip(range(3), data_list, titles):
    sns.barplot(x=data.iloc[:,0], y=data.iloc[:,1], ax=axs[i])
    axs[i].set_title(titles[i], fontsize=20)
    axs[i].tick_params(labelsize=14)
    axs[i].set_xlabel(x_labels[i], fontsize=16)
    axs[i].set_ylabel('출고량', fontsize=16)
plt.show()

# 최근 10년간 많이 출고된 게임


*   확인을 해보니 어쌔신크리드게임이 1등이고 2등 3등이 슈팅게임이여서 결과가 슈팅게임이 1등인것같다.
*   하지만 최근 10년간 1등인 게임이 액션 어드벤쳐게임이지만 액션게임분류가 되므로
*   플스 플랫폼이고 액션게임인것을 다음분기에 게임을 설계를 해야한다.
*  





In [ ]:
plt.figure(figsize=(30, 16))
a = sns.barplot(x = year_sales_top_game.index, y='Global_Sales', data=year_sales_top_game)
i = 0
for name, val in zip(year_sales_top_game['Name'], year_sales_top_game['Global_Sales']):
    if val >= 7:
        a.text(x=i, y=val-1, s=(name), ha='center',va='top', fontsize=28, rotation=90)
    else:
        a.text(x=i, y=val+1, s=(name), ha='center', fontsize=28, rotation=90)
    i+=1
a.set_title('최근 10년의 출고량 1위 게임', fontsize=30)
plt.yticks(fontsize=20)    
plt.xticks(ticks=[x for x in range(10)], labels=[str(x)+' 위' for x in range(1, 11)], fontsize=16)
plt.show()